In [1]:
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd


import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_file = pd.read_csv("../TAMU_FINAL_DATASET_2018.csv")

In [3]:
# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

In [4]:
df.shape

(100000, 469)

In [5]:
sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)


df = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)
# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(2726, 469)
(2726, 469)
(5452, 469)
(4040, 469)
(4040, 468)
(4040,)


In [6]:
# modeling 

x = tf.placeholder(dtype = tf.float32, shape = [None, 468])
y = tf.placeholder(dtype = tf.int32, shape = [None])


# tanh, elu, selu, softplus, and softsign

#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.relu)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.tanh)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.selu)


#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.softplus)
logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.elu)


loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_pred = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("x: ", x)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", correct_pred)

x:  Tensor("Placeholder:0", shape=(?, 468), dtype=float32)
logits:  Tensor("fully_connected/Elu:0", shape=(?, 128), dtype=float32)
loss:  Tensor("Mean:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [7]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
        #print('EPOCH', i)
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: X_train, y: y_train})
        if i % 50 == 0:
            print("Loss: ", loss)
        #print('DONE WITH EPOCH')

Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)


In [8]:

predicted = sess.run([correct_pred], feed_dict={x: X_train})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_train, predicted)])

# Calculate the accuracy
accuracy = match_count / len(X_train)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.493


In [9]:


predicted = sess.run([correct_pred], feed_dict={x: X_test})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])

# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.510


In [30]:


from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)



pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1.0
0.6079460269865068


In [10]:
# try some other sklearn model here
from sklearn.svm import SVC

clf = SVC(kernel='rbf', gamma='auto')
clf.fit(X_train, y_train)

print('finish fitting data')


pred_train = clf.predict(X_train)

pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))



finish fitting data
1.0
0.4917541229385307


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(n_estimators=100, max_depth=2, criterion='entropy', random_state=0)
clf.fit(X_train, y_train)



print(clf.feature_importances_)


pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

[0.00289259 0.0039376  0.00043894 0.05731346 0.07990643 0.
 0.         0.00102756 0.0033593  0.         0.00153819 0.
 0.0033882  0.         0.         0.00064987 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00036454 0.00179876 0.         0.         0.
 0.         0.         0.         0.         0.         0.00403502
 0.00795568 0.00023798 0.         0.00140605 0.00753876 0.
 0.         0.         0.         0.         0.00087898 0.
 0.         0.         0.         0.         0.         0.
 0.         0.00059974 0.00142143 0.0048986  0.00288857 0.
 0.         0.         0.00437133 0.00243041 0.0097332  0.0063828
 0.00412831 0.00677299 0.00581217 0.00341202 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00578643 0.         0.         0.00171221 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00106067 0.         0.
 0.         0.00081173 0.00255651 0.     

In [14]:
important = clf.feature_importances_
extractedIdx = np.nonzero(important)

In [21]:
print(len(extractedIdx[0]))
extractedIdx

107


(array([  0,   1,   2,   3,   4,   7,   8,  10,  12,  15,  25,  26,  35,
         36,  37,  39,  40,  46,  55,  56,  57,  58,  62,  63,  64,  65,
         66,  67,  68,  69,  79,  82,  93,  97,  98, 100, 105, 118, 134,
        139, 142, 143, 145, 146, 148, 149, 151, 152, 154, 158, 159, 161,
        170, 174, 175, 219, 221, 291, 296, 297, 298, 299, 300, 301, 304,
        305, 306, 307, 310, 312, 313, 319, 322, 323, 332, 334, 336, 346,
        378, 379, 380, 381, 382, 383, 387, 388, 390, 391, 393, 395, 396,
        397, 403, 409, 410, 412, 414, 416, 417, 419, 423, 424, 426, 434,
        436, 437, 458]),)

In [26]:
df.columns[0]
extracted_name = []
for idx in extractedIdx:
    extracted_name.append(df.columns[idx])

print(len(extracted_name[0]))
lst = list(extracted_name[0])
newDF = df[lst].copy()

107


In [29]:
newDF.describe()

,ID,AGE,AMI_FLAG,ORIG_REAS_ENTITLE_CD,RECON_MA_RISK_SCORE_NBR,CON_VISIT_04_Q02,CON_VISIT_04_Q03,CON_VISIT_21_Q01,CON_VISIT_21_Q03,CON_VISIT_05_Q02,...,Index_Health_ins_engage,Index_Health_ins_influence,Population_density_centile_US,CDC_NPH_GAP,CDC_EYE_GAP,BCS_GAP,ACE_ELIG,STATIN_ELIG,SEX_CD_F,Diab_Type_Diabetes Type I
count,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,...,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000
mean,50063.227475,74.558416,0.493069,0.269802,1.375350,0.536386,0.512376,0.843317,0.832426,0.105446,...,2.356436,3.028465,58.892079,0.016089,0.104950,0.051238,0.490594,0.476733,0.517822,0.036634
std,28931.487039,9.302996,0.500014,0.447799,1.114341,1.871897,1.545656,2.667597,2.426516,0.720561,...,2.176692,2.385259,25.943360,0.125834,0.306528,0.220509,0.499973,0.499520,0.499744,0.187884
min,14.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24925.500000,69.000000,0.000000,0.000000,0.640000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,50392.500000,74.000000,0.000000,0.000000,1.032000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,3.000000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,75201.500000,81.000000,1.000000,1.000000,1.735500,1.000000,1.000000,0.000000,0.000000,0.000000,...,3.000000,5.000000,82.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,99978.000000,95.000000,1.000000,3.000000,11.385000,65.000000,28.000000,51.000000,35.000000,13.000000,...,9.000000,9.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:


# use only the 107 features, retrain models

df = newDF
print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(4040, 107)
(4040, 107)
(4040, 106)
(4040,)


In [97]:
# modeling 

x = tf.placeholder(dtype = tf.float32, shape = [None, 106])
y = tf.placeholder(dtype = tf.int32, shape = [None])


# tanh, elu, selu, softplus, and softsign

#logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.elu)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.tanh)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.selu)


# this two model has better accuracy -------------

logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.relu)
logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.softplus)



loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_pred = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("x: ", x)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", correct_pred)

x:  Tensor("Placeholder_34:0", shape=(?, 106), dtype=float32)
logits:  Tensor("fully_connected_19/Softplus:0", shape=(?, 50), dtype=float32)
loss:  Tensor("Mean_34:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax_17:0", shape=(?,), dtype=int64)


In [98]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
        #print('EPOCH', i)
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: X_train, y: y_train})
        if i % 50 == 0:
            print("Loss: ", loss)
        #print('DONE WITH EPOCH')

Loss:  Tensor("Mean_34:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_34:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_34:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_34:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_34:0", shape=(), dtype=float32)


In [99]:

predicted = sess.run([correct_pred], feed_dict={x: X_train})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_train, predicted)])

# Calculate the accuracy
accuracy = match_count / len(X_train)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.531


In [100]:


predicted = sess.run([correct_pred], feed_dict={x: X_test})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])

# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.543


In [66]:



clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)



pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1.0
0.5472263868065967


In [67]:
clf = SVC(kernel='rbf', gamma='auto')
clf.fit(X_train, y_train)

print('finish fitting data')


pred_train = clf.predict(X_train)

pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

finish fitting data
1.0
0.49250374812593706


In [68]:

clf = RandomForestClassifier(n_estimators=100, max_depth=2, criterion='entropy', random_state=0)
clf.fit(X_train, y_train)



print(clf.feature_importances_)


pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

[0.00076343 0.05192551 0.         0.0930434  0.         0.00184984
 0.         0.         0.00063242 0.         0.00456323 0.00211605
 0.00164865 0.02511586 0.         0.         0.         0.
 0.         0.         0.00845447 0.00108493 0.00187023 0.
 0.00320354 0.0075394  0.0053885  0.0064146  0.01584575 0.00153646
 0.         0.00065686 0.         0.00652982 0.         0.0035972
 0.         0.         0.00401855 0.         0.00411384 0.
 0.00542829 0.         0.00822809 0.00042279 0.0100366  0.00033866
 0.         0.03737866 0.         0.         0.         0.
 0.         0.         0.         0.         0.00577827 0.00539838
 0.02637311 0.         0.00060563 0.00439411 0.00314754 0.0341734
 0.00311063 0.00032447 0.         0.00405149 0.         0.00041658
 0.02125726 0.00024506 0.         0.         0.         0.
 0.13021067 0.10675221 0.04512861 0.04430996 0.10976912 0.
 0.02583011 0.00121981 0.03333855 0.00080374 0.         0.03203068
 0.02298011 0.         0.         0.00056882 